In [1]:
import config
from post_process.utils import load_data, preprocess_data, process_result, get_noise_S, result_process
import scipy.io as scio
from torch import tensor, no_grad, sqrt, float32
from model.model import DReaM_net
import numpy as np
np.random.seed(40)
def process_data_and_plot(Type, is_save=True, version="last.ckpt"):

    data = load_data(Type)
    hard, rate, sc_type, var_rate, expand = result_process(Type)
    test_input = preprocess_data(data['NmrData'], sc_type)
    
    S = test_input/test_input[:,:,0:1]

    S = tensor(S, dtype=float32)
    with no_grad():
        module = DReaM_net.load_from_checkpoint(f"Result/DOSY/{version}")
        module.prepare_for_testing()
        test_out, var = module(S)
    test_out, var = test_out.squeeze(), sqrt(var).squeeze()
    test_input = test_input/np.max(test_input[:, :, 0], 1)
    test_input_scaling = test_input[:, :, 0][:, :, np.newaxis].squeeze(0)
    test_out = (test_out * test_input_scaling / config.mul_label).cpu().numpy()
    var = (var * test_input_scaling/ config.mul_label).cpu().numpy()
    out_threshold = np.max(test_out, axis=1, keepdims=True) * rate
    test_out[test_out < out_threshold] = 0
    threshold = np.max(var, axis=1, keepdims=True) * var_rate
    var[var < threshold] = 0
    
    test_out[test_out < np.max(test_out) * hard] = 0
    var[var < np.max(var)*0.02] = 0

    spec_var, max_var, merged_boxes = process_result(test_out, data['ppm'], var, data['idx_peaks'], data['HNMR'], expand)
    if is_save:
        scio.savemat("./MATLAB/Result/" + Type + "_Result.mat", {"Z":test_out, "var":spec_var, "max_var":max_var,  "box":merged_boxes})

for Type in ["QGC", "GSP", "M6", "QG", "FPH", "MPD", "BPP1", "BPP2", "AMO", "AD"]:
    process_data_and_plot(Type)

In [3]:
import config
from post_process.utils import  process_result
import scipy.io as scio
from torch import tensor, no_grad, float32
from model.model import DReaM_net
import numpy as np

if config.Type == 'T1T2':
    Type = 'T2'
    NmrData = scio.loadmat("./Dataset/" + Type + "_net_input.mat")

    input = NmrData['S']
    HNMR = NmrData['HNMR']
    test_input_scaling = (HNMR[0]/np.max(HNMR[0]))[:,np.newaxis]

    with no_grad():
        module = DReaM_net.load_from_checkpoint("Result/T1T2/last.ckpt")
        module.prepare_for_testing()

    test_out = np.zeros([NmrData['S'].shape[0], module.hparams.label_size])
    test_var = np.zeros([NmrData['S'].shape[0], module.hparams.label_size])

    test_input = input.reshape(NmrData['S'].shape[0],1,-1)
    test_input = tensor(test_input).to(float32)
    out, var = module(test_input)

    test_out = out.squeeze(1).cpu().numpy()
    var = var.squeeze(1).sqrt().cpu().numpy()

    test_out[test_out < np.max(test_out) * 0.7] = 0
    test_out = (test_out * test_input_scaling)
    var = (var * test_input_scaling)

    out_threshold = np.max(test_out, axis=1, keepdims=True) * 0.9
    test_out[test_out < out_threshold] = 0
    threshold = np.max(var, axis=1, keepdims=True) * 0.6
    var[var < threshold] = 0

    spec_var, max_var, merged_boxes = process_result(test_out, NmrData['ppm'], var, NmrData['idx_peaks'], input[:,0:1], 0, 0, 5)
    scio.savemat("./MATLAB/Result/" + Type + "_Result.mat", {"Z":test_out, "var":spec_var, "max_var":max_var,  "box":merged_boxes})